In [ ]:
# Incase Of Update
response = requests.get('https://db.ygoprodeck.com/api/v7/cardinfo.php')
json_response = response.json()
dataset = pd.DataFrame(json_response['data'])

dataset.to_csv('Dataset/Yugioh_Database.csv')

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings

import random as random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


warnings.simplefilter(action='ignore', category=FutureWarning)
plt.style.use('Solarize_Light2')
pd.set_option('display.max_columns', 20)

#import requests
#import itertools

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)


In [57]:
# ========================================================================
'''Load Dataset'''
dataset = pd.read_csv('Dataset/Yugioh_Database.csv')
dataset = dataset.drop(['Unnamed: 0' , 'frameType' , 'archetype' , 'ygoprodeck_url' , 'card_sets' , 'card_images' , 'card_prices' , 'banlist_info'],axis=1)
dataset = dataset[dataset['type'] != 'XYZ Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Synchro Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Fusion Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Pendulum Normal Monster']

dataset = dataset[dataset['type'] != 'Skill Card']
dataset = dataset[dataset['type'] != 'Monster Token']

dataset.loc[dataset['type']=='Normal Monster', ['desc']] = 'NoInfo'
dataset = dataset.fillna('0')
dataset['level'] = dataset['level'].astype('int32')



# ========================================================================
'''Create Tokenized sequence database'''


df = dataset['desc']         #Tokenizer is only trained on desc and based on that . Otherwise if trained on names it would blow vocab up to absurd amounts
Sliced_df = dataset[['level' , 'race' , 'type' , 'attribute' , 'name' , 'desc']]

for i in range(1,11,2):
    Sliced_df.insert(loc=i, column='A'+str(i), value=-1)        # Adds seperator columns


Sliced_df = Sliced_df.reset_index(drop=True)            # Need to reset the indexes so they are consistent
df = df.reset_index(drop=True)                              

tokenizer = Tokenizer(filters='\r , \n , \" ') # Speech marks stop names from being recognised by tokenizer
tokenizer.fit_on_texts(df)
tokenizer.word_index['0'] = 0           #Signifies Empty values
tokenizer.word_index['-1'] = -1           #Signifies Seperators

sequences = []
padded_sequences = []
Tokenized_sequence_database = []
count = 0
for i in Sliced_df.astype('string').to_numpy():
    
    sequences.append(tokenizer.texts_to_sequences(i))
    

for i in range(0,11):
    padded_sequences.append( pad_sequences(np.array(sequences , dtype='object')[:,i], padding='post') ) 

Tokenized_sequence_database = np.concatenate(([padded_sequences[i] for i in range(11)]) , axis=1 )

    




In [187]:
def Deck_Loader(directory):
    '''Loads Decks from Deck_Lists.txt as arrays and stores those arrays in altered'''
    file = open(directory , 'r')
    read = file.readlines()
    Deck_Array = []
    flag = False

    temp=[]

    for count,line in enumerate(read):
        
        if '//' in read[count]:
            flag = not flag
        
        if flag:
            
            read[count] = read[count].replace('\n','')
            
            if ('=='  in read[count]) or ('//'  in read[count])  :
                pass
                
            else:
                for i in range(int(read[count][0])):
                    temp.append(read[count][1:].strip())          #skip appending also remove white space

        if (not flag) or (count == len(read) - 1):
            Deck_Array.append(temp)
            temp = []
            flag = not flag
            
            
    file.close()
    return Deck_Array 


def stitcher(Deck_Index , Deck_Array):
    '''Picks 5 random cards from a certain deck in a deck array and stitches them together'''
    
    decider = [random.choice(Deck_Array[Deck_Index]) for _ in range(5)]
    output = np.concatenate(([Tokenized_sequence_database[i] for i in Sliced_df[Sliced_df['name'].isin(decider)].index.values]) )
    if len(output) != 905:
        return stitcher(Deck_Index , Deck_Array)
    else:
        return output

def random_no_match_generator(length):
    '''Generated Datapoints which correspond to a card that doesnt relate to a given group of decider cards'''
    out = []
    for _ in range(length):
        temp = []
        subject_card = np.concatenate( ( random.choice(Tokenized_sequence_database)  , np.zeros(724)) , axis=None )
        decider_cards = np.concatenate([random.choice(Tokenized_sequence_database) for _ in range(5)])

        temp.append(decider_cards)
        temp.append(subject_card)
        temp.append([0])

        out.append(temp)
    
    return out

def Dataset_Builder(directory , include_no_match):
    '''Builds a dataset with some specificaiton. This could be a training/validation dataset or a experimentation dataset'''
    altered = Deck_Loader(directory)
    Built_Dataset = []
    for deck_index,deck in enumerate(altered):
        
        for card_index in Sliced_df[Sliced_df['name'].isin(deck)].index.values:
            
            subject_card =  np.concatenate( (Tokenized_sequence_database[card_index] , np.zeros(724)) , axis=None ) # Extends subject to equal length of decider cards
            for _ in range(2):
                temp = []
                decider_cards = stitcher(deck_index , altered)
                temp.append(decider_cards)
                temp.append(subject_card)
                temp.append([1])
                Built_Dataset.append(temp)


    if (include_no_match):
        Built_Dataset.extend( random_no_match_generator(958 ) )
    random.shuffle(Built_Dataset)
    random.shuffle(Built_Dataset)
    random.shuffle(Built_Dataset)
    return Built_Dataset



Training_Validation_Dataset = Dataset_Builder('Dataset/Deck_Lists.txt' , True)
pd.DataFrame(Training_Validation_Dataset)


,0,1,2
0,"[0, -1, 59, 0, -1, 80, 5, 0, 0, -1, 0, -1, 566...","[332.0, -1.0, 246.0, 0.0, -1.0, 21.0, 8.0, 0.0...",[0]
1,"[114, -1, 371, 0, -1, 21, 8, 0, 0, -1, 138, -1...","[273.0, -1.0, 371.0, 0.0, -1.0, 21.0, 8.0, 0.0...",[1]
2,"[0, -1, 99, 0, -1, 117, 8, 0, 0, -1, 138, -1, ...","[97.0, -1.0, 274.0, 0.0, -1.0, 21.0, 8.0, 0.0,...",[0]
3,"[97, -1, 286, 147, -1, 21, 8, 0, 0, -1, 138, -...","[0.0, -1.0, 329.0, 0.0, -1.0, 80.0, 5.0, 0.0, ...",[0]
4,"[277, -1, 99, 0, -1, 21, 8, 0, 0, -1, 138, -1,...","[0.0, -1.0, 59.0, 0.0, -1.0, 112.0, 5.0, 0.0, ...",[0]
...,...,...,...
1911,"[114, -1, 246, 0, -1, 21, 8, 0, 0, -1, 138, -1...","[0.0, -1.0, 109.0, 0.0, -1.0, 80.0, 5.0, 0.0, ...",[0]
1912,"[0, -1, 59, 0, -1, 112, 5, 0, 0, -1, 0, -1, 2,...","[97.0, -1.0, 254.0, 0.0, -1.0, 21.0, 8.0, 0.0,...",[0]
1913,"[0, -1, 59, 0, -1, 112, 5, 0, 0, -1, 0, -1, 18...","[0.0, -1.0, 329.0, 0.0, -1.0, 80.0, 5.0, 0.0, ...",[0]
1914,"[332, -1, 364, 0, -1, 21, 8, 0, 0, -1, 198, -1...","[0.0, -1.0, 147.0, 0.0, -1.0, 117.0, 8.0, 0.0,...",[0]


In [193]:
# create a experimentation decklist with same format and push it to builder to build out a dataset. Put sequences into the sequence to text converters to see relation table with names.
# See how accurate model is at predicting card relations withing a deck -> It should predict most cards as 1.

# Create a function which allows you to type 5 cards and the create a dataset with every other card in the game as a subject and see what cards the model predicts will go well with your chosen cards.

Experimentation_Dataset = Dataset_Builder('Dataset/Experimental_Deck_Lists.txt' , False)
pd.DataFrame(Experimentation_Dataset)    

,0,1,2
0,"[0, -1, 59, 0, -1, 112, 5, 0, 0, -1, 0, -1, 39...","[61.0, -1.0, 364.0, 0.0, -1.0, 21.0, 8.0, 0.0,...",[1]
1,"[332, -1, 303, 0, -1, 21, 8, 0, 0, -1, 202, -1...","[0.0, -1.0, 59.0, 0.0, -1.0, 80.0, 5.0, 0.0, 0...",[1]
2,"[114, -1, 371, 0, -1, 21, 8, 0, 0, -1, 138, -1...","[61.0, -1.0, 902.0, 0.0, -1.0, 170.0, 8.0, 0.0...",[1]
3,"[0, -1, 59, 0, -1, 112, 5, 0, 0, -1, 0, -1, 32...","[61.0, -1.0, 411.0, 0.0, -1.0, 75.0, 8.0, 0.0,...",[1]
4,"[273, -1, 364, 0, -1, 59, 8, 0, 0, -1, 138, -1...","[7.0, -1.0, 364.0, 0.0, -1.0, 379.0, 21.0, 8.0...",[1]
...,...,...,...
269,"[0, -1, 59, 0, -1, 80, 5, 0, 0, -1, 0, -1, 172...","[0.0, -1.0, 41.0, 0.0, -1.0, 80.0, 5.0, 0.0, 0...",[1]
270,"[114, -1, 1158, 0, -1, 21, 8, 0, 0, -1, 202, -...","[114.0, -1.0, 303.0, 0.0, -1.0, 21.0, 8.0, 0.0...",[1]
271,"[7, -1, 286, 147, -1, 21, 8, 0, 0, -1, 138, -1...","[0.0, -1.0, 274.0, 0.0, -1.0, 117.0, 8.0, 0.0,...",[1]
272,"[0, -1, 59, 0, -1, 112, 5, 0, 0, -1, 0, -1, 12...","[332.0, -1.0, 371.0, 0.0, -1.0, 93.0, 8.0, 0.0...",[1]


In [224]:
#tokenizer.sequences_to_texts([Training_Validation_Dataset[542][1]])
m , m2 = next(Experimentation_Gen())
l1, l2 = m
l1

<tf.Tensor: shape=(1, 905), dtype=float64, numpy=
array([[  0.,  -1.,  59.,   0.,  -1.,  80.,   5.,   0.,   0.,  -1.,   0.,
         -1., 318.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  -1.,
         14.,  16.,   7.,  46., 194.,  15., 187.,  59.,   8.,  11.,   3.,
         57.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 

In [61]:
def positional_encoding(length, depth):
  depth = depth/2
  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate( [np.sin(angle_rads), np.cos(angle_rads)], axis=-1) 

  return pos_encoding

class Positional_Embedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model ):
    super().__init__()

    self.d_model = d_model
    
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=False) 
    self.pos_encoding = positional_encoding(length=905, depth=d_model)
    
  def call(self, x):
    x = self.embedding(x)
    
    x*= np.sqrt(self.d_model) # Scale Values by their embedding dimensionality otherwise they could get overwhelmed by positional encoder
    x = x + self.pos_encoding 
    return x





class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class DeciderSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, ffn, dropout_rate):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(ffn, activation='relu'), 
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x
  
class EncoderLayer(tf.keras.layers.Layer):
  """Single Encoder Layer with DeciderMHA and Feed Forward layer"""
  def __init__(self, d_model, ffn , dropout_rate ):
    super().__init__()

    self.DSA = DeciderSelfAttention(num_heads=4, key_dim=100)       # Scaling Number of Heads increases parameters as this is a different implementation of mha compared to attention is all you need paper.
    self.FF = FeedForward(d_model, ffn , dropout_rate)

  def call(self, x):
    
    
    x = self.DSA(x)

    x = self.FF(x)

    return x

class Encoder(tf.keras.layers.Layer):
  """Full Encoder with embedding layer with dropout and encoder layers"""
  def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.num_layers = num_layers

    self.Pos_Embedding = Positional_Embedding(vocab_size, d_model)
    self.EL = [EncoderLayer(d_model, ffn , dropout_rate) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self,x):
    x = self.Pos_Embedding(x)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.EL[i](x)

    return x



class SubjectSelfAttention(BaseAttention):

  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class SubjectCrossAttention(BaseAttention):

  def call(self, x , context):
    attn_output = self.mha(
        query=x,
        value=context,
        key=context)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class ComparatorLayer(tf.keras.layers.Layer):

  def __init__(self , d_model, ffn , dropout_rate):
    super().__init__()

    self.SSA = SubjectSelfAttention(num_heads=4, key_dim=100)      
    self.SCA = SubjectCrossAttention(num_heads=4, key_dim=100)
    self.FF = FeedForward(d_model, ffn , dropout_rate)

  def call(self, x , context):
    
    x = self.SSA(x)

    x = self.SCA(x , context)

    x = self.FF(x)

    return x
  
class Comparator(tf.keras.layers.Layer):
  
  def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.num_layers = num_layers

    self.Pos_Embedding = Positional_Embedding(vocab_size, d_model)
    self.CL = [ComparatorLayer(d_model, ffn , dropout_rate) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x , context):
    x = self.Pos_Embedding(x)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.CL[i](x , context)

    return x

class FinalFeedForward(tf.keras.layers.Layer):
  def __init__(self , dropout_rate):
    super().__init__()

    self.seq = tf.keras.Sequential([
      tf.keras.layers.Flatten(),          #Flattens sentances for each card comparision , into a single 1d array , so it can generate probabilities properly, instead of shoving 100 x905 matrix straight through and generating 100 probabilities for each card comparision feature embedding
      tf.keras.layers.Dense(50, activation='relu'),
      tf.keras.layers.Dense(25, activation='relu'),
      tf.keras.layers.Dropout(dropout_rate),
      tf.keras.layers.Dense(1 , activation='sigmoid')
      
    ])
    
  def call(self, x):
    
    x = self.seq(x) 
    return x
  
class FullModel(tf.keras.Model):
   def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.enc = Encoder(d_model, vocab_size , ffn , dropout_rate , num_layers)
    self.com = Comparator(d_model, vocab_size , ffn , dropout_rate , num_layers)
    self.FFF = FinalFeedForward(dropout_rate)

   def call(self, inputs):
     context , x = inputs
     
     
     
     context = self.enc(context)
     x = self.com(x , context)

     x = self.FFF(context)

     return x
   
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [194]:

def Train_Gen():
    seti0 = [i[0] for i in Training_Validation_Dataset]
    seti1 = [i[1] for i in Training_Validation_Dataset]
    seti2 = [i[2] for i in Training_Validation_Dataset]
    for _ in range(2000):
        r_int = random.randint(0 , 1700)
        train_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+30])
        train_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+30])
        train_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+30])
    
        yield (train_seti0,train_seti1), train_seti2

def Val_Gen():
    seti0 = [i[0] for i in Training_Validation_Dataset]
    seti1 = [i[1] for i in Training_Validation_Dataset]
    seti2 = [i[2] for i in Training_Validation_Dataset]
    for _ in range(330):
        r_int = random.randint(1730 , 1860)
        val_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+50])
        val_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+50])
        val_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+50])

        yield (val_seti0,val_seti1), val_seti2

def Experimentation_Gen():
    seti0 = [i[0] for i in Experimentation_Dataset]
    seti1 = [i[1] for i in Experimentation_Dataset]
    seti2 = [i[2] for i in Experimentation_Dataset]

    for _ in range(len(Experimentation_Dataset)):
        # For some reason i doesnt iterate and gets stuck at 0 , so had to use random int
        r_int = random.randint(0 , 90)
        exp_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+1])
        exp_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+1])
        exp_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+1])

        yield (exp_seti0,exp_seti1), exp_seti2



In [190]:
#Testing accuracy is much higher than training accuracy. This is due to dropouts causing lower accuracy during training but giving a more robust model when testing
Model = FullModel(100 , 12647 , 1000 , 0.2 , 2)

learning_rate = CustomSchedule(d_model = 100)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

Model.compile(
    loss= tf.keras.losses.BinaryCrossentropy(),
    optimizer=optimizer,
    metrics= 'accuracy' )

history = Model.fit(Train_Gen() , epochs=20, 
                               validation_data = Val_Gen()  , steps_per_epoch=20 , batch_size=30 , validation_steps=10 , validation_batch_size=50)

Epoch 1/20
20/20 [==============================] - 8s 299ms/step - loss: 0.7707 - accuracy: 0.5000 - val_loss: 0.7082 - val_accuracy: 0.5740
Epoch 2/20
20/20 [==============================] - 5s 251ms/step - loss: 0.7081 - accuracy: 0.5000 - val_loss: 0.6861 - val_accuracy: 0.5460
Epoch 3/20
20/20 [==============================] - 5s 248ms/step - loss: 0.6817 - accuracy: 0.5717 - val_loss: 0.6384 - val_accuracy: 0.7580
Epoch 4/20
20/20 [==============================] - 5s 246ms/step - loss: 0.6549 - accuracy: 0.6317 - val_loss: 0.6241 - val_accuracy: 0.5720
Epoch 5/20
20/20 [==============================] - 5s 256ms/step - loss: 0.6230 - accuracy: 0.6583 - val_loss: 0.5003 - val_accuracy: 0.8140
Epoch 6/20
20/20 [==============================] - 5s 265ms/step - loss: 0.5877 - accuracy: 0.6817 - val_loss: 0.5019 - val_accuracy: 0.7940
Epoch 7/20
20/20 [==============================] - 5s 271ms/step - loss: 0.5377 - accuracy: 0.7333 - val_loss: 0.4472 - val_accuracy: 0.8120
Epoch 

In [76]:

k

(<tf.Tensor: shape=(1, 905), dtype=int32, numpy=
 array([[  61,   -1,  364,    0,   -1,   21,    8,    0,    0,   -1,  138,
           -1, 7793,  320,    0,    0,    0,    0,    0,    0,    0,   -1,
           12,    4,    5,   13,   91,   27,    7,   34,    5,   32,    2,
           41,   25,    1,    6,  152,   17,   80,  228, 1505,  152,    7,
           80,  228,   32,   25,  195,   12,    4,    5,   13,   52,   28,
          290,    1,    6,   14,   16,    7,   46,   61,   15,  127,  364,
            8,   11,    3,   31,   22,  191,   81,  125,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0, 

In [195]:
pred = pd.DataFrame(Model.predict(Experimentation_Gen()))
count = 0
for i in pred[0]:
    if i > 0.5:
        count+=1
print(count/len(pred))


274/274 [==============================] - 2s 7ms/step
0.4635036496350365


In [199]:
k , trash = next(Experimentation_Gen())
l1 , l2 = k
singular_pred = Model.predict(k)
singular_pred

1/1 [==============================] - 0s 24ms/step


array([[0.10321745]], dtype=float32)

In [210]:

tokenizer.sequences_to_texts([l1.numpy()])

TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.

[<tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=-1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=59>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=-1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=112>,
 <tf.Tensor: shape=(), dtype=int32, numpy=5>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=-1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=-1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=392>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1185>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(),